<a href="https://colab.research.google.com/github/hirokiOS/SentimentAnalysisWithDownloadedDataSource/blob/main/2_sentiment_analysis_finetuning_finbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 大規模言語モデルのファインチューニング

DATAMODEは次の３項のいずれかからご選択ください。
DATAMODE == 'WRIME' : 感情分析のデータセット。モデルは東北大学の日本語LLM. 
DATAMODE == 'ACCERN-API' : URLとAPIを指定してACCERNのAPIを用いる。
DATAMODE == 'ACCERN-DRIVE' : ノートブック１においてURLとAPIを指定して保存したデータを用いる。

In [ ]:
# DATAMODE='WRIME'
# DATAMODE='ACCERN-DRIVE' use dataset preprocessed in 1_original_accern_parse.ipynb
DATAMODE='ACCERN-API' # directly donload dataset from accern provided API

if DATAMODE == 'ACCERN-API':
    accern_api_url = '< Vender provided URL should come here >'
    accern_token_url = '< Vendor provided token should come here >'

In [ ]:
from google.colab import drive
drive.mount("drive")
colab_path = "drive/MyDrive/Colaboratory/"

**注意**
こちらのノートブックは、感情分析のデータセットを用いてコードを実行するために用意されています。

# 2 感情分析モデルの実装

### 2.1 環境の準備

In [ ]:
# Testing following on 3.11.3
!pip install --upgrade pip

In [ ]:
#!pip uninstall -y numpy # reset numpy for dependency
#!pip uninstall -y setuptools
!pip install setuptools
!pip install numpy

# pytorch installation information found here
# https://pytorch.org/get-started/locally/
# GPU environment
!pip install torch torchvision torchaudio
# CPU environment
#!pip3 install torch torchvision torchaudio                                                                   # on mac
#!pip install torch==2.0.0+cpu torchvision==0.6.0+cpu -f https://download.pytorch.org/whl/torch_stable.html  # on windows


In [ ]:
# Transformers and Japanese releated environments
!pip install "transformers[ja,torch]" # transformers==4.24?
!pip install  datasets matplotlib japanize-matplotlib tqdm
# dependents of pre-trained model
!pip install fugashi ipadic unidic-lite torchsummary torchtext sentencepiece
!pip install ipywidgets
!pip install accern-data==0.0.4

In [ ]:

if DATAMODE == 'ACCERN-API':
    import requests
    # Save datagenerators as file to colab working directory
    # If you are using GitHub, make sure you get the "Raw" version of the code
    giturl1 = 'https://raw.githubusercontent.com/hirokiOS/SentimentAnalysisWithDownloadedDataSource/main/acc_function/ACCDFConcatenator.py'
    giturl2 = 'https://raw.githubusercontent.com/hirokiOS/SentimentAnalysisWithDownloadedDataSource/main/acc_function/ConvertDfToHFData.py'
    
    giturls = [giturl1, giturl2]
    
    for giturl in giturls:
        r = requests.get(giturl)
        
        filename = giturl.split("/")[-1]
        # make sure your filename is the same as how you want to import 
        with open(filename, 'w') as f:
            f.write(r.text)
    
    # now we can import
    from ACCDFConcatenator import DFConcatenator
    from ConvertDfToHFData import Convert4SentimentAnalysis

乱数を固定

In [ ]:
import torch
from transformers.trainer_utils import set_seed
# from transformers import set_seed

# 乱数シードを42に固定
set_seed(42)

### 2.2 データセットの準備
Accern API経由のデータを利用する場合、利用可能なurlとtokenの指定をお願い致します。

In [ ]:
from pprint import pprint
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import pickle

if DATAMODE=='WRIME':
    # Hugging Face Hub上のllm-book/wrime-sentimentのリポジトリから
    # データを読み込む
    train_dataset = load_dataset("llm-book/wrime-sentiment", split="train", remove_neutral=False)
    valid_dataset = load_dataset("llm-book/wrime-sentiment", split="validation", remove_neutral=False)
    # pprintで見やすく表示する

elif DATAMODE in ['ACCERN-DRIVE', 'ACCERN-API']:
    # Note book 1 で作成したDataframeのPickleファイルを読み込み学習用のデータセットとする。
    if DATAMODE == 'ACCERN-DRIVE':
        with open(colab_path + 'data/accern_dataset.pkl', 'rb') as f:
            accern_dataset = pickle.load(f)

    # APIより取得したデータを加工しHugging faceのファインチューニングに用いる。
    elif DATAMODE == 'ACCERN-API':
        df = DFConcatenator(
                        url=accern_api_url,
                        token=accern_token_url, 
                        start_date="2016-09-08", end_date="2016-09-09", 
                        output_pattern="oct31", output_path="./accern_raw_json/", 
                        mode = "json", split_dates=False)

        accern_dataset = Convert4SentimentAnalysis(df, threshold = 20, target_label = 'event_sentiment')


    train_valid_dataset = accern_dataset.train_test_split(test_size=0.10)
    # required_labels = ['sentence', 'label', 'datetime']
    train_dataset = train_valid_dataset['train'] # [required_labels]
    valid_dataset = train_valid_dataset['test']  #[required_labels]

print(train_dataset[3])


In [ ]:
print(len(train_valid_dataset['train']))
print(len(train_valid_dataset['test']))

In [ ]:
# pprint(train_dataset.features)

### 2.3 トークナイザ

In [ ]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    PreTrainedModel,
)

# specify model name on Hugging Face Hub

if DATAMODE == 'WRIME':
    model_name = "cl-tohoku/bert-base-japanese-v3"
    # model_name = "cl-tohoku/bert-base-japanese-whole-word-masking",
    tokenizer = AutoTokenizer.from_pretrained(model_name) # read tokenizer from model name
    print(tokenizer.tokenize("これはテストです。"))
    encoded_input = tokenizer("これはテストです。")

elif DATAMODE in ['ACCERN-DRIVE', 'ACCERN-API']:
    model_name = "ProsusAI/finbert"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    print(tokenizer.tokenize("This is a test for Finbert"))
    encoded_input = tokenizer("This is a test for Finbert")

# check tokenizer class
print(type(tokenizer).__name__)

In [ ]:
# 出力されたオブジェクトのクラスを表示
print(type(encoded_input).__name__)

In [ ]:
pprint(encoded_input)

In [ ]:
tokenizer.convert_ids_to_tokens(encoded_input["input_ids"])

### 2.4 データセット統計の可視化

In [ ]:
from collections import Counter
import japanize_matplotlib
import matplotlib.pyplot as plt
from datasets import Dataset
from tqdm import tqdm

plt.rcParams["font.size"] = 18  # 文字サイズを大きくする

def visualize_text_length(dataset: Dataset):
    """データセット中のテキストのトークン数の分布をグラフとして描画"""
    # データセット中のテキストの長さを数える
    length_counter = Counter()
    for data in tqdm(dataset):
        length = len(tokenizer.tokenize(data["sentence"]))
        length_counter[length] += 1
    # length_counterの値から棒グラフを描画する
    plt.bar(length_counter.keys(), length_counter.values(), width=5.0)
    plt.xlabel("トークン数")
    plt.ylabel("事例数")
    plt.show()

visualize_text_length(train_dataset)
visualize_text_length(valid_dataset)

In [ ]:
def visualize_labels(dataset: Dataset):
    """データセット中のラベル分布をグラフとして描画"""
    # データセット中のラベルの数を数える
    label_counter = Counter()
    for data in dataset:
        label_id = data["label"]
        label_name = dataset.features["label"].names[label_id]
        label_counter[label_name] += 1
    # label_counterを棒グラフとして描画する
    plt.bar(label_counter.keys(), label_counter.values(), width=1.0)
    plt.xlabel("ラベル")
    plt.ylabel("事例数")
    plt.show()

visualize_labels(train_dataset)
visualize_labels(valid_dataset)

### 2.5 データセットの前処理

In [ ]:
from transformers import BatchEncoding

def preprocess_text_classification(
    example: dict[str, str | int]
) -> BatchEncoding:
    """文書分類の事例のテキストをトークナイズし、IDに変換"""
    encoded_example = tokenizer(example["sentence"], max_length=512, truncation = True)
    # モデルの入力引数である"labels"をキーとして格納する
    encoded_example["labels"] = example["label"]
    return encoded_example

In [ ]:
train_dataset

トレイニングと検証に用いるデータサイズの最大量をSETTRAINMAXとSETVALIDMAXで指定してください。

Specify the number of maximum data used if just testing the code. 
larger number might be only available on local GPU environment

In [ ]:
## reduce data size in prototyping
SETTRAINMAX = 3000
SETVALIDMAX = 1000

TRAINSIZE = min(SETTRAINMAX, len(train_dataset))
VALIDSIZE = min(SETVALIDMAX, len(valid_dataset))

train_dataset_subset = train_dataset.select(range(TRAINSIZE))
valid_dataset_subset = valid_dataset.select(range(VALIDSIZE))


encoded_train_dataset = train_dataset_subset.map(
    preprocess_text_classification,
    remove_columns=train_dataset.column_names,
)
encoded_valid_dataset = valid_dataset_subset.map(
    preprocess_text_classification,
    remove_columns=valid_dataset.column_names,
)

In [ ]:
print(encoded_train_dataset[0])

### 2.6 ミニバッチ構築

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

batch_inputs = data_collator(encoded_train_dataset[0:4])
pprint({name: tensor.size() for name, tensor in batch_inputs.items()})

### 2.7 モデルの準備

In [ ]:
from transformers import AutoModelForSequenceClassification

class_label = train_dataset.features["label"]
label2id = {label: id for id, label in enumerate(class_label.names)}
id2label = {id: label for id, label in enumerate(class_label.names)}
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=class_label.num_classes,
    label2id=label2id,  # ラベル名からIDへの対応を指定
    id2label=id2label,  # IDからラベル名への対応を指定
)
print(type(model).__name__)

In [ ]:
print(model.forward(**data_collator(encoded_train_dataset[0:4])))

### 2.8 訓練の実行

In [ ]:
from transformers import TrainingArguments
import torch
use_cuda = torch.cuda.is_available()
print('USE CUDA : ' + str(use_cuda))


training_args = TrainingArguments(
    output_dir="output_finbert",  # 結果の保存フォルダ
    per_device_train_batch_size=16,  # 訓練時のバッチサイズ
    per_device_eval_batch_size=16,  # 評価時のバッチサイズ
    learning_rate=2e-5,  # 学習率
    lr_scheduler_type="linear",  # 学習率スケジューラの種類
    warmup_ratio=0.1,  # 学習率のウォームアップの長さを指定
    num_train_epochs=5,  # エポック数
    save_strategy="epoch",  # チェックポイントの保存タイミング
    logging_strategy="epoch",  # ロギングのタイミング
    evaluation_strategy="epoch",  # 検証セットによる評価のタイミング
    load_best_model_at_end=True,  # 訓練後に開発セットで最良のモデルをロード
    metric_for_best_model="accuracy",  # 最良のモデルを決定する評価指標
    fp16=use_cuda,  # 自動混合精度演算の有効化 # cuda only option
)

In [ ]:
import numpy as np

def compute_accuracy(
    eval_pred: tuple[np.ndarray, np.ndarray]
) -> dict[str, float]:
    """予測ラベルと正解ラベルから正解率を計算"""
    predictions, labels = eval_pred
    # predictionsは各ラベルについてのスコア
    # 最もスコアの高いインデックスを予測ラベルとする
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_valid_dataset,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_accuracy,
)
trainer.train()

### 2.9 訓練後のモデルの評価

In [ ]:
# 検証セットでモデルを評価
eval_metrics = trainer.evaluate(encoded_valid_dataset)
pprint(eval_metrics)


In [ ]:
# Train 前のモデル評価
# device = torch.device("cpu")
# print(model.to(device).forward(**data_collator(encoded_train_dataset[0:4])))
# seq_class_valid_out = model.to(device).forward(**data_collator(encoded_valid_dataset[0:]))

### 2.10 モデルの保存

Google Driveへの保存

In [ ]:
# Googleドライブをマウントする
from google.colab import drive

drive.mount("drive")

In [ ]:
# 保存されたモデルをGoogleドライブのフォルダにコピーする
!mkdir -p drive/MyDrive/Colaboratory/data
!cp -r output_finbert drive/MyDrive/Colaboratory/data

In [ ]:
# checkpointに加え、現在の状態も保存する。
model_dir = '/content/drive/My Drive/Colaboratory/data/'
trainer.save_model(model_dir + 'finbert_finetuned/model')

Hugging Face Hubへの保存 (今回は非推奨, NDAの範囲外のデータで保存したい場合。)

In [ ]:
# from huggingface_hub import login
# 
# login()
# # Hugging Face Hubのリポジトリ名
# # "YOUR-ACCOUNT"は自らのユーザ名に置き換えてください
# repo_name = "YOUR-ACCOUNT/bert-base-japanese-v3-wrime-sentiment"
# # トークナイザとモデルをアップロード
# tokenizer.push_to_hub(repo_name)
# model.push_to_hub(repo_name)